## Private State (Estado privado)
Primeiro, vamos abordar o caso da passagem de estado privado entre nós.

Isso é útil para qualquer coisa necessária como parte da lógica de funcionamento intermediária do grafo, mas não é relevante para a entrada ou saída geral do grafo.

In [ ]:
from  typing_extensions import TypedDict
from IPython.display import Image, display
from langgraph.graph import StateGraph, START, END

class OverState(TypedDict):
    foo: str

class PrivateState(TypedDict):
    baz: int

def node_1(state: OverState) -> PrivateState:
    print("----- Node 1 -----")
    return {"baz": state["foo"] + 1}

def node_2(state: PrivateState) -> OverState:
    print("----- Node 2 -----")
    return {"foo": state["baz"] + 1}

builder= StateGraph(OverState)

builder.add_node("node_1", node_1)
builder.add_node("node2", node_2)

builder.add_edge(START, "node_1")
builder.add_edge("node_1", "node_2")
builder.add_edge("node_2", END)

graph= builder.compile()

display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
graph.invoke({"foo": 1})

## Input/Output Schema (Esquema de Entrada/Saída)
Por padrão, o StateGraph aceita um único esquema e espera-se que todos os nós se comuniquem com esse esquema.

No entanto, também é possível definir esquemas de entrada e saída explícitos para um grafo.

Nesses casos, geralmente definimos um esquema "interno" que contém todas as chaves relevantes para as operações do grafo.

Mas usamos esquemas de entrada e saída específicos para restringir a entrada e a saída.

In [ ]:
class OverallState(TypedDict):
    question: str
    answer: str
    notes: str

def thinking_node(state: OverallState):
    return {"answer": "bye", "notes": "... his name is Lance"}

def answer_node(state:OverallState):
    return {"answer": "bye Lance"}

builder= StateGraph(OverallState)
builder.add_node("thinking_node", thinking_node)
builder.add_node("answer_node", answer_node)
builder.add_edge(START, "thinking_node")
builder.add_edge("thinking_node", "answer_node")
builder.add_edge("answer_node", END)

graph= graph.compile()
display(Image(graph.get_graph().draw_mermaid_png()))

Agora, vamos usar um esquema de entrada e saída específico com nosso grafo.

Aqui, os esquemas de entrada/saída filtram quais chaves são permitidas na entrada e na saída do grafo.

Além disso, podemos usar uma dica de tipo `InputState` para especificar o esquema de entrada de cada um dos nossos nós.

Isso é importante quando o grafo usa vários esquemas.

Usamos dicas de tipo abaixo para, por exemplo, mostrar que a saída de `answer_node` será filtrada para `OutputState`.

In [ ]:
class InputState(TypedDict):
    questoin: str

class OutputState(TypedDict):
    answer: str

class OverallState(TypedDict):
    question: str
    answer: str
    notes: str

def thinking_node(state: InputState):
    return {"answer": "bye", "noter": "... his name is Lance"}

def answer_node(state: OverallState) -> OutputState:
    return {"answer": "bye Lance"}

# A saída só tera um dicionário com "answer" pois foi definido no StateGraph no output OutputState que só possui answer
builder= StateGraph(OverallState, input= InputState, output= OutputState)
graph.add_node("answer_node", answer_node)
graph.add_node("thinking_node", thinking_node)
graph.add_edge(START, "thinking_node")
graph.add_edge("thinking_node", "answer_node")
graph.add_edge("answer_node", END)

graph = graph.compile()

# View
display(Image(graph.get_graph().draw_mermaid_png()))

graph.invoke({"question":"hi"})